# Getting Started

In [1]:
import pandas as pd
import numpy as np

In [2]:
filepath="../data/preprocessed_data/train.csv"

In [3]:
df=pd.read_csv(filepath,index_col=0)

In [4]:
housing = df.drop("median_house_value", axis=1)

In [5]:
housing.columns

Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'ocean_proximity'],
      dtype='object')

# Custom Transformer
- Let's start with building our custom transformer

In [5]:
from sklearn.base import BaseEstimator, TransformerMixin
rooms_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6

In [6]:
class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room = True):
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):
        return self # nothing else to do
    def transform(self, X, y=None):
        rooms_per_household = X[:, rooms_ix] / X[:, households_ix]
        population_per_household = X[:, population_ix] / X[:, households_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household,
             bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]

In [7]:
attr_adder = CombinedAttributesAdder(add_bedrooms_per_room=False)
housing_extra_attribs = attr_adder.transform(housing.values)

# Feature Scaling 
- This is another important task in transformation we need to scale the values
- There are two common ways to get all attributes to have the same scale: **min-max scaling and standardization**

In [8]:
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# Transformation Piplelines

## Numerical Column Pipeline

In [9]:
housing_num =housing.drop('ocean_proximity',axis=1)

In [10]:
from sklearn.pipeline import Pipeline

In [11]:
num_pipeline=Pipeline([
    ('imputer',SimpleImputer(strategy='median')),
    ('attrib_adder',CombinedAttributesAdder()),
    ('scaler',StandardScaler())
    
])

In [12]:
housing_num_tr=num_pipeline.fit(housing_num)

## Column Transformer

In [13]:
from sklearn.compose import ColumnTransformer
num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]

In [14]:
full_pipeline=ColumnTransformer([
    ("num",num_pipeline,num_attribs),
    ("cat",OneHotEncoder(),cat_attribs)
])

In [15]:
housing_prepared=full_pipeline.fit_transform(housing)

In [16]:
import joblib
joblib.dump(full_pipeline, '../models/preprocessing_pipeline.pkl')

['../models/preprocessing_pipeline.pkl']

In [17]:
import os
path = os.getcwd()

In [18]:
path

'D:\\Developer\\End-to-End-Ml-Project\\notebooks'